 # TEAM 7 FOR THE WIN  - Projet Micromania, à l'abordage des jeux vidéos

In [ ]:
workon team7_projet 
pip install selenium, bs4, pandas, pymongo, requests

Site choisi : https://www.micromania.fr/jeux.html

In [44]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

Fonction pour compléter notre Dataframe

In [41]:
def append_df(soup, df_final):
    image = []
    title = []
    console = []
    price = []
    date_de_sortie = []
    genre = []
    details = []
    df = pd.DataFrame(columns=['ImageLink', 'Title', 'Console', 'Price', 'Année_de_sortie', 'Genre', 'Details'])

    try: 
        images = [image['src'] for image in soup.findAll('img',{'itemprop': 'image'}, src=True)]
        image.append(images[0])
    except BaseException:
        image.append('Missing')
    try: 
        title.append(soup.select_one("h1[class*= pdp-product-name]").findChildren()[0].text.strip())
    except BaseException:
        title.append('Missing')
    try: 
        console.append(soup.select_one("h1[class*= pdp-product-name]").findChildren()[1].text.strip())
    except BaseException:
        console.append('Missing')
    try: 
        price.append(float(soup.select_one("span[class=value]").attrs['content']))
    except BaseException:
        price.append(0.0)
    try: 
        date_de_sortie.append(int(soup.select_one("ul[class*=list-unstyled]").findChildren()[2].text.strip()[-4:]))
    except BaseException:
        date_de_sortie.append(0)
    try: 
        genre.append(soup.select_one("ul[class*=list-unstyled]").findChildren()[5].text.strip())
    except BaseException:
        genre.append('Missing')
    try: 
        details.append(soup.select_one("article[class=product-details__description]").text.strip())
    except BaseException:
        details.append('Missing')
  
    df.loc[:, 'ImageLink'] = image
    df.loc[:, 'Title'] = title
    df.loc[:, 'Console'] = console
    df.loc[:, 'Price'] = price
    df.loc[:, 'Année_de_sortie'] = date_de_sortie
    df.loc[:, 'Genre'] = genre
    df.loc[:, 'Details'] = details
    df_final = pd.concat([df_final, df])

    return df_final

Fonction pour récupérer les éléments de chaque produits

In [42]:
def iterate_over_products(soup, df_final):
    # Récupère tout les hrefs relatifs des produits de la page 
    all_products = soup.find_all('a', {'class': 'product-image-link'}, href=True)

    # Concaténer avec le chemin absolu dans une list comprehensions
    href_game = ['https://www.micromania.fr' + a['href'] for a in all_products]

    # Boucle sur le nombre d'élement de la page
    for i in range(len(href_game)):
        soupProduct = BeautifulSoup(requests.get(href_game[i]).content)
        df_final = append_df(soupProduct, df_final)
    
    return df_final

On récupère le nombre d'éléments

In [68]:
soup = BeautifulSoup(requests.get('https://www.micromania.fr/jeux.html').content)
max_elements = int(soup.select_one('.show-more-text').text[26:].strip().replace(' ', ''))
#max_elements = 200
max_elements

1660

On boucle sur toutes les pages du site 

In [47]:
request = requests.get(f'https://www.micromania.fr/jeux.html?start=0&sz=48')
position = 0
df_final = pd.DataFrame(columns=['ImageLink', 'Title', 'Console', 'Price', 'Année_de_sortie', 'Genre', 'Details'])
while position <= max_elements:
    soup = BeautifulSoup(requests.get(f'https://www.micromania.fr/jeux.html?start={position}&sz=48').content)
    df_final = iterate_over_products(soup, df_final)
    position += 48

df_final

,ImageLink,Title,Console,Price,Année_de_sortie,Genre,Details
0,https://www.micromania.fr/dw/image/v2/BCRB_PRD...,Sea Of Thieves,XBOX ONE,19.99,2018,Action,Sea of Thieves est un jeu multijoueur d'un tou...
0,https://www.micromania.fr/dw/image/v2/BCRB_PRD...,Forza Horizon 4,XBOX ONE,39.99,2018,Course / arcade,Forza Horizon 4 est le très attendu successeur...
0,https://www.micromania.fr/dw/image/v2/BCRB_PRD...,Gears 5,XBOX ONE,14.99,2019,Wargame,Issus d’une des sagas les plus populaires de l...
0,https://www.micromania.fr/dw/image/v2/BCRB_PRD...,Ori And The Will Of The Wisps,XBOX ONE,14.99,2020,Action / aventure,Un nouveau voyage commence.Embarquez dans une ...
0,https://www.micromania.fr/dw/image/v2/BCRB_PRD...,Minecraft Dungeons Hero Edition,XBOX ONE,29.99,2020,Activités récréatives,Frayez-vous un chemin à travers un tout nouvea...
...,...,...,...,...,...,...,...
0,https://www.micromania.fr/dw/image/v2/BCRB_PRD...,Pac-man & Galaga Dimensions,3DS,12.99,0,Missing,Pac-Man et Galaga sont deux jeux mythiques des...
0,https://www.micromania.fr/dw/image/v2/BCRB_PRD...,Call Of Duty Black Ops Cold War,PS5,64.99,0,Missing,La saga iconique Black Ops est de retour avec ...
0,https://www.micromania.fr/dw/image/v2/BCRB_PRD...,Godfall,PS5,59.99,0,Missing,Aperion est au bord du précipice.Vous êtes le ...
0,https://www.micromania.fr/dw/image/v2/BCRB_PRD...,Halo Infinite,XBOX SERIES X,0.00,0,Missing,DISPONIBLE EN 2021 : Lorsque tout espoir est p...


On récupère notre dataframe que l'on transforme en json

In [66]:
import json
df_final = df_final.reset_index(drop=True)
df_final['Année_de_sortie'] = df_final['Année_de_sortie'].astype(int)
df_final.to_json("./games.json", orient='records')

df_final.drop(['Details'], axis=1, inplace=True) # Pour Power BI
df_final.to_csv("./games.csv", sep="§") # Pour Power BI !
df_final.dtypes

ImageLink           object
Title               object
Console             object
Price              float64
Année_de_sortie      int32
Genre               object
dtype: object

Création du client pymongo pour mongodb

In [61]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['micromania']
games = db['games']

Insertion des datas dans mongodb

In [62]:
with open("games.json") as f:
    data_json = json.load(f)
games.insert_many(data_json)